In [1]:
import pickle
import pandas as pd
import numpy as np
import optuna

#     optuna_ex1_hyperparameters_per_dataset.py
#  -> analysis_ex1_hyperparameters.ipynb
#  -> benchmark_ex1_best_hyperparameters.py
#  -> analysis_ex1_hyperparameters_best.ipynb

In [2]:
SETUP = "FG"  # "FG" or "2D"
storage = "sqlite:///optuna_databases/optuna_ex1_parameter_tuning_{}_e3_d2.sqlite3".format(SETUP)  

In [3]:
studies = optuna.study.get_all_study_summaries(storage=storage)

In [4]:
loaded_study = optuna.load_study(study_name=studies[0].study_name, storage=storage)
df = loaded_study.trials_dataframe()
for key, value in loaded_study.system_attrs.items():
    df[key] = value
for study in studies[1:]:
    loaded_study = optuna.load_study(study_name=study.study_name , storage=storage)
    df_new = loaded_study.trials_dataframe()
    for key, value in loaded_study.system_attrs.items():
        df_new[key] = value
    df = pd.concat([df, df_new])
df.reset_index(inplace=True, drop=True)

In [5]:
df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [6]:
df.head(1)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,0,0.740102,2021-08-21 17:48:24.568239,2021-08-21 17:48:28.238238,0 days 00:00:03.669999,213.239295,0.885378,2,3,least_squares_linear_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.initialization_method.value_counts()

BGM_init         4500
Kmeans_init      4500
str              4500
KDTmeans_init    4500
Name: initialization_method, dtype: int64

In [8]:
np.unique(df.initialization_method)

array(['BGM_init', 'KDTmeans_init', 'Kmeans_init', 'str'], dtype=object)

In [9]:
np.unique(df["Data X"])

array(['abalone_input.pd', 'banknote_input.pd', 'breast_cancer_input.np',
       'cars_input.pd', 'contraceptive_input.pd', 'generated6_input.np',
       'iris_input.pd', 'steel_input.pd', 'students_input.pd'],
      dtype=object)

In [5]:
top = 10
df_best = pd.DataFrame()
for dataset in np.unique(df["Data X"]):
    df_new = df[df["Data X"] == dataset].sort_values(by=["user_attrs_accuracy_val"], ascending=False).head(top)
    df_best = pd.concat([df_best, df_new])
df_best.reset_index(inplace=True, drop=True)


In [6]:
df_best.shape

(90, 27)

In [12]:
df_best.head(2)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,178,0.745530,2021-08-21 17:59:04.674246,2021-08-21 17:59:08.223247,0 days 00:00:03.549001,218.298073,0.906463,2,3,least_squares_linear_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191,0.742337,2021-08-21 17:59:51.088247,2021-08-21 17:59:54.543247,0 days 00:00:03.455000,194.335065,0.928138,2,3,least_squares_linear_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_{}_e3_d2.pd".format(SETUP), "wb"))

In [14]:
df.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,249.5,0.719596,222.925837,0.947941,2.0,3.0,False,0.746928,0.719596,0.009362,0.015042,6.701391,0.323159,0.160758,0.802553,0.708743,0.0
banknote_input.pd,249.5,0.991176,229.954405,0.969802,2.0,3.0,False,0.996089,0.991176,0.001994,0.005344,7.191966,0.138767,0.133596,0.474706,0.517996,0.0
breast_cancer_input.np,249.5,0.913666,131.539937,0.859962,2.0,3.0,False,0.974125,0.913666,0.006132,0.022664,4.821376,0.332732,0.329695,0.692122,0.558487,0.0
cars_input.pd,249.5,0.847312,141.483992,0.924894,2.0,3.0,False,0.861921,0.847312,0.017721,0.022846,8.699622,0.373897,0.061730,0.652574,0.786946,0.0
contraceptive_input.pd,249.5,0.483805,148.817029,0.899313,2.0,3.0,False,0.543067,0.483805,0.020660,0.027870,8.067238,0.183961,0.232810,0.496521,0.671042,0.0
generated6_input.np,249.5,0.938525,211.294302,0.962373,2.0,3.0,False,0.944703,0.938525,0.012274,0.015016,5.666505,0.149210,0.164795,0.632582,0.310539,0.0
iris_input.pd,249.5,0.936393,165.939946,0.869258,2.0,3.0,False,0.992095,0.936393,0.006006,0.037854,3.299346,0.393391,0.211619,0.575003,0.654293,0.0
steel_input.pd,249.5,0.637949,165.084491,0.950437,2.0,3.0,False,0.694806,0.637949,0.020242,0.024788,8.393757,0.188497,0.074822,0.568162,0.734654,0.0
students_input.pd,249.5,0.445045,153.707300,0.900974,2.0,3.0,False,0.553837,0.445045,0.018120,0.048763,2.737378,0.260569,0.076004,0.518480,0.590814,0.0


In [15]:
df_best.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,267.3,0.741890,218.681779,0.924778,2.0,3.0,False,0.756971,0.741890,0.004482,0.010979,7.649212,0.469216,0.157457,0.850881,0.903901,0.0
banknote_input.pd,249.4,1.000000,255.778928,0.989243,2.0,3.0,False,0.999968,1.000000,0.000056,0.000000,8.532366,0.154428,0.101675,NaN,NaN,NaN
breast_cancer_input.np,238.8,0.942019,113.828319,0.855472,2.0,3.0,False,0.971521,0.942019,0.005947,0.018506,3.789244,0.283436,0.289642,0.650309,0.557668,0.0
cars_input.pd,252.7,0.915895,151.114653,0.940625,2.0,3.0,False,0.928678,0.915895,0.004659,0.014425,NaN,NaN,NaN,NaN,NaN,NaN
contraceptive_input.pd,207.5,0.535960,141.825572,0.893146,2.0,3.0,False,0.582790,0.535960,0.013578,0.024868,8.518477,0.105458,0.209386,NaN,NaN,NaN
generated6_input.np,283.8,0.974534,248.255777,0.969441,2.0,3.0,False,0.975964,0.974534,0.001833,0.004459,5.277966,0.242609,0.150597,0.649288,0.295004,0.0
iris_input.pd,273.6,0.974107,150.499792,0.818059,2.0,3.0,False,0.986905,0.974107,0.007055,0.021256,NaN,NaN,NaN,0.463354,0.669457,0.0
steel_input.pd,222.3,0.683993,68.547552,0.918679,2.0,3.0,False,0.726368,0.683993,0.014456,0.020408,8.897039,0.162168,0.044104,NaN,NaN,NaN
students_input.pd,173.2,0.522229,154.567738,0.919654,2.0,3.0,False,0.540209,0.522229,0.013866,0.038928,1.469243,0.319357,0.071491,NaN,NaN,NaN


In [16]:
df["params_use_2_dim_gate_based_on"].value_counts(dropna=False)

NaN    18000
Name: params_use_2_dim_gate_based_on, dtype: int64

In [17]:
df_best["params_optimization_method"].value_counts(dropna=False)

least_squares_linear_regression    42
ridge_regression                   34
lasso_regression                   14
Name: params_optimization_method, dtype: int64

In [18]:
df_best.groupby(["params_optimization_method"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
params_optimization_method,,,,,,,,,,,,,,,,,
lasso_regression,226.785714,0.745629,161.253901,0.868855,2.0,3.0,False,0.758890,0.745629,0.011176,0.028967,1.469243,0.319357,0.071491,0.399801,0.641888,0.0
least_squares_linear_regression,232.214286,0.793853,145.037677,0.916286,2.0,3.0,False,0.817868,0.793853,0.007414,0.015885,8.125587,0.204707,0.073771,0.634556,0.444811,0.0
ridge_regression,257.588235,0.856636,196.525413,0.930676,2.0,3.0,False,0.874083,0.856636,0.005632,0.013693,7.798381,0.159208,0.163735,0.724703,0.514131,0.0


In [19]:
df_best.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [26]:
df_best.groupby(["initialization_method"]).count()

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,state,Data X,Data y,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
initialization_method,,,,,,,,,,,,,,,,,,,,,
BGM_init,16,16,16,16,16,16,16,16,16,16,...,16,16,16,0,0,0,16,16,16,16
KDTmeans_init,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,0,0,0,0
Kmeans_init,12,12,12,12,12,12,12,12,12,12,...,12,12,12,0,0,0,0,0,0,0
str,35,35,35,35,35,35,35,35,35,35,...,35,35,35,0,0,0,0,0,0,0


In [7]:
df_best.groupby(["initialization_method"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
initialization_method,,,,,,,,,,,,,,,,,
BGM_init,263.375000,0.951925,161.273382,0.904457,2.0,3.0,False,0.962860,0.951925,0.004914,0.012868,NaN,NaN,NaN,0.604038,0.515743,0.0
KDTmeans_init,250.518519,0.768688,173.111633,0.928021,2.0,3.0,False,0.793053,0.768688,0.009241,0.017041,7.22845,0.195539,0.116834,NaN,NaN,NaN
Kmeans_init,183.083333,0.778093,122.626197,0.873666,2.0,3.0,False,0.796533,0.778093,0.007397,0.025797,NaN,NaN,NaN,NaN,NaN,NaN
str,243.171429,0.788107,180.145673,0.922259,2.0,3.0,False,0.809061,0.788107,0.006926,0.016077,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_best["params_weight_concentration_prior_type"].value_counts()

dirichlet_distribution    14
dirichlet_process          2
Name: params_weight_concentration_prior_type, dtype: int64

In [21]:
df_best[df_best["params_weight_concentration_prior_type"] == "dirichlet_distribution"][['params_mean_precision_prior',
       'params_weight_concentration_prior']].mean()

params_mean_precision_prior          0.626263
params_weight_concentration_prior    0.497032
dtype: float64

In [22]:
df_best[df_best["params_weight_concentration_prior_type"] == "dirichlet_distribution"][['params_mean_precision_prior',
       'params_weight_concentration_prior']].median()

params_mean_precision_prior          0.679215
params_weight_concentration_prior    0.437203
dtype: float64

In [23]:
df_best[['params_alpha', 'params_beta','params_gamma']].mean()

params_alpha    7.228450
params_beta     0.195539
params_gamma    0.116834
dtype: float64

In [24]:
df_best[['params_alpha', 'params_beta','params_gamma']].median()

params_alpha    8.520954
params_beta     0.196226
params_gamma    0.099398
dtype: float64